In [1]:
#!/usr/bin/env python
# # -*- coding: utf-8 -*-
#!/usr/bin/env python
#!pip install tensorflow==1.15

import numpy as np
import matplotlib.pyplot as plt
import pickle
import glob, os, inspect
import argparse
import time
import seaborn as sns
# import matplotlib
# matplotlib.use("Qt5Agg")
# #matplotlib.use('Agg')
# from mpl_toolkits.mplot3d import axes3d
# import matplotlib.pyplot as plt
import pandas as pd
import sys,inspect
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Postprocess")
sys.path.append("C:\\Users\\Vinamr J\\Documents\\qic\\qrc\\hqr\\hqr-master\\chaos\\Methods\\Models\\Utils")

import utils  # for utils.py
from utils import *
import global_utils  # for global_utils.py

# import utils
# from utils import *
# import sys
# sys.path.append('../Methods/Models/Utils')
# import global_utils

In [2]:
sysname, tidx = "SST", 2
file_path = os.path.abspath("GRU_analysis.ipynb")
current_directory = os.path.dirname(file_path)
mid_directory = os.path.dirname(current_directory)
parent_directory = os.path.dirname(mid_directory)

print("Current Directory:", current_directory)
print("Parent Directory:", parent_directory)

#current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
results_dir = os.path.join(os.path.dirname(mid_directory), "Results")  # Use os.path.join for path concatenation
print(results_dir)

eval_path = os.path.join(results_dir, sysname, 'Evaluation_Data')
print(eval_path)

model_path = os.path.join(results_dir, sysname, 'Trained_Models')
print(model_path)

fig_path = os.path.join(results_dir, sysname, 'Eval_Figures')
if not os.path.isdir(fig_path):
    os.makedirs(fig_path)  # Use os.makedirs to create parent directories if needed
print(fig_path)

Current Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Data\SST
Parent Directory: c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Evaluation_Data
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Trained_Models
c:\Users\Vinamr J\Documents\qic\qrc\hqr\hqr-master\chaos\Results\SST\Eval_Figures


In [4]:
models = []
samples = []
size_values = [80,100,120,200,500,1000]
layer_values = [1,2,3]
for size in size_values:
    for layers in layer_values:
        string1 = f"RNN-lstm-RDIM_5-N_used_427-NLAY_{layers}-SLAY_{size}-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3"
        string2 = f"LSTM-{size}-({layers})"
        models.append([os.path.join(eval_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [3]:
# models = [[os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_20-IUL_0-REG_1e-07-AU_0-NICS_3"),"HQR-5,V-10,DL=40,IPL=20,n_tests=3"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_8-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_8-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=8,IPL=8,n_tests=1"],
#           [os.path.join(eval_path, "hqrc_pinv-RDIM_5-N_used_427-DL_40-Nqr_5-A_0.9-J_2.0-fJ_1-V_10-NL_0-IPL_300-IUL_0-REG_1e-07-AU_0-NICS_1"),"HQR-5,V-10,DL=40,IPL=300,n_tests=1"]]
# samples = ["HQR-5,V-10,DL=40,IPL=20,n_tests=3","HQR-5,V-10,DL=8,IPL=8,n_tests=1","HQR-5,V-10,DL=40,IPL=300,n_tests=1"]
# title = "Sea Surface Temperatures"

In [6]:
rmse_dict = dict()
targets = dict()
outputs = dict()
sp_outputs = dict()
sp_targets = dict()
Wouts = dict()
coeffs = dict()
rmnse_avg_train_dict = dict()
rmnse_avg_test_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'results.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            qs = QResults()
            qs.rmnse_avg_test = rs['rmnse_avg_TEST']
            qs.rmnse_avg_train = rs['rmnse_avg_TRAIN']
            qs.n_pred_005_avg_test = rs['num_accurate_pred_005_avg_TEST']
            qs.n_pred_005_avg_train = rs['num_accurate_pred_005_avg_TRAIN']
            qs.n_pred_050_avg_test = rs['num_accurate_pred_050_avg_TEST']
            qs.n_pred_050_avg_train = rs['num_accurate_pred_050_avg_TRAIN']
            qs.model_name = rs['model_name']
            #if qs.rmnse_avg_test != np.inf and qs.rmnse_avg_train != np.inf:
                #print(rs.keys())
            #print(qs.model_name)
            #print('train={}, test={}'.format(qs.rmnse_avg_train, qs.rmnse_avg_test))
            #qs.info()
            rmnse_avg_train_dict[label] = rs['rmnse_avg_TRAIN']
            rmnse_avg_test_dict[label] = rs['rmnse_avg_TEST']
            pred_test = rs['predictions_all_TEST']
            truth_test = rs['truths_all_TEST']
            print(pred_test.shape)
            M = len(pred_test)
            print('{} Number of test'.format(qs.model_name), M)
            rmsels = []
            for j in range(M):
                    rmsels.append(calNRMSE(pred_test[j], truth_test[j]))

            rmse_dict[label] = np.mean(np.array(rmsels), axis=0)
            print(rmse_dict[label].shape)


            targets[label] = truth_test[0]
            outputs[label] = pred_test[0]

            # For frequency
            sp_outputs[label] = rs['sp_pred_TEST']
            sp_targets[label] = rs['sp_true_TEST']
    else:
        print('Not found {}'.format(fname))
    # if trained_models is not None:
    #     rfolder, label = trained_models[i][0], trained_models[i][1]
    #     fname = os.path.join(rfolder, 'data.pickle')
    #     if os.path.isfile(fname):
    #         print('File existed: ', fname)
    #         with open(fname, 'rb') as rfile:
    #             #try:
    #             rs = pickle.load(rfile)
    #             #except:
    #             #    continue
    #             print(rs.keys())
    #             coeffs[label] = np.array(rs['coeffs'])
    #             Wouts[label]  = rs['W_out'][:-1].reshape((coeffs[label].shape[0], -1))
    #             print(Wouts[label].shape, coeffs[label].shape)
    #     else:
    #         print('Not found saved model {}'.format(fname))

(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_1-SLAY_80-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_2-SLAY_80-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_3-SLAY_80-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_1-SLAY_100-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_2-SLAY_100-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_3-SLAY_100-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3 Number of test 3
(300,)
(3, 300, 5)
RNN-lstm-RDIM_5-N_used_427-NLAY_1-SLAY_120-ISH_st

In [8]:
model_test_error = []
test_avg_error = []
for label in samples:
    model_test_error.append([label,np.mean(rmse_dict[label])])
    test_avg_error.append(np.mean(rmse_dict[label]))
print(model_test_error)
print("Test Average error = ",np.mean(test_avg_error))

[['LSTM-80-(1)', 1.2064266672027109], ['LSTM-80-(2)', 1.3706204527702754], ['LSTM-80-(3)', 1.271992163299086], ['LSTM-100-(1)', 1.1392373863340042], ['LSTM-100-(2)', 1.2740005100570893], ['LSTM-100-(3)', 1.5606296716914485], ['LSTM-120-(1)', 1.214182357485659], ['LSTM-120-(2)', 1.2146606175946864], ['LSTM-120-(3)', 1.141039929488017], ['LSTM-200-(1)', 1.4037708264604738], ['LSTM-200-(2)', 1.4260363909213802], ['LSTM-200-(3)', 1.2551532189298598], ['LSTM-500-(1)', 1.2780587100457148], ['LSTM-500-(2)', 1.2411615217131895], ['LSTM-500-(3)', 1.5192277422883036], ['LSTM-1000-(1)', 1.228413423491027], ['LSTM-1000-(2)', 1.1684160821837812], ['LSTM-1000-(3)', 1.3567768080345202]]
Test Average error =  1.2927669155550683


In [9]:
print(rmnse_avg_train_dict)
print(rmnse_avg_test_dict)
error_array_test = []
error_array_train = []
for label in samples:
    error_array_train.append(rmnse_avg_train_dict[label])
    error_array_test.append(rmnse_avg_test_dict[label])
print("Train average error for GRU =", np.mean(error_array_train))
print("Test average error for GRU =", np.mean(error_array_test))

{'LSTM-80-(1)': 1.0835170348853467, 'LSTM-80-(2)': 1.1383941722228825, 'LSTM-80-(3)': 1.2244220310672396, 'LSTM-100-(1)': 1.1055570807241213, 'LSTM-100-(2)': 1.081033925670105, 'LSTM-100-(3)': 1.1688340787631635, 'LSTM-120-(1)': 1.0636765405772628, 'LSTM-120-(2)': 1.052611618915613, 'LSTM-120-(3)': 1.1994498573841859, 'LSTM-200-(1)': 1.0730366846329902, 'LSTM-200-(2)': 1.1506426338687556, 'LSTM-200-(3)': 1.0697734186519672, 'LSTM-500-(1)': 1.0221435577868427, 'LSTM-500-(2)': 1.0487135397096272, 'LSTM-500-(3)': 1.1070923596884787, 'LSTM-1000-(1)': 1.1094575169668393, 'LSTM-1000-(2)': 1.0020061245034115, 'LSTM-1000-(3)': 1.156503322952313}
{'LSTM-80-(1)': 1.1168952996830417, 'LSTM-80-(2)': 1.25862186091868, 'LSTM-80-(3)': 1.094394954992666, 'LSTM-100-(1)': 1.0617729196857966, 'LSTM-100-(2)': 1.1750297306505932, 'LSTM-100-(3)': 1.3087033223706652, 'LSTM-120-(1)': 1.0305169708737922, 'LSTM-120-(2)': 1.105690010757122, 'LSTM-120-(3)': 1.0066879040514716, 'LSTM-200-(1)': 1.190308439452362, '

In [10]:
models = []
samples = []
size_values = [80,100,120,200,500,1000]
layer_values = [1,2,3]
for size in size_values:
    for layers in layer_values:
        string1 = f"RNN-lstm-RDIM_5-N_used_427-NLAY_{layers}-SLAY_{size}-ISH_statefull-SL_16-PL_4-LR_0.001-DKP_1.0-ZKP_1.0-HSPL_70-IPL_300-NL_0-NICS_3"
        string2 = f"LSTM-{size}-({layers})"
        models.append([os.path.join(model_path,string1),string2])
        samples.append(string2)
title = "Sea Surface Temperatures"

In [11]:
train_time_dict = dict()
memory_dict = dict()
trainable_parameters_dict = dict()
for i in range(len(models)):
    rfolder, label = models[i][0], models[i][1]
    fname = os.path.join(rfolder, 'data.pickle')
    if os.path.isfile(fname):
        with open(fname, 'rb') as rfile:
            try:
                rs = pickle.load(rfile)
            except:
                continue
            #print(rs.keys())
            train_time_dict[label]=rs["total_training_time"]
            trainable_parameters_dict[label]=rs["n_trainable_parameters"]
            memory_dict[label]=rs["memory"]
    else:
        print('Not found {}'.format(fname))

In [12]:
train_time_dict

{'LSTM-80-(1)': 1.1061666011810303,
 'LSTM-80-(2)': 1.653533697128296,
 'LSTM-80-(3)': 2.486395835876465,
 'LSTM-100-(1)': 0.9051737785339355,
 'LSTM-100-(2)': 1.9125862121582031,
 'LSTM-100-(3)': 2.441983222961426,
 'LSTM-120-(1)': 1.3475751876831055,
 'LSTM-120-(2)': 2.0844483375549316,
 'LSTM-120-(3)': 2.736746311187744,
 'LSTM-200-(1)': 1.083221197128296,
 'LSTM-200-(2)': 2.013763904571533,
 'LSTM-200-(3)': 2.82649302482605,
 'LSTM-500-(1)': 2.1218972206115723,
 'LSTM-500-(2)': 3.1633200645446777,
 'LSTM-500-(3)': 4.3087968826293945,
 'LSTM-1000-(1)': 2.0235180854797363,
 'LSTM-1000-(2)': 3.896742582321167,
 'LSTM-1000-(3)': 5.911282062530518}

In [13]:
memory_dict

{'LSTM-80-(1)': 330.54296875,
 'LSTM-80-(2)': 345.2109375,
 'LSTM-80-(3)': 359.703125,
 'LSTM-100-(1)': 333.171875,
 'LSTM-100-(2)': 347.03515625,
 'LSTM-100-(3)': 362.63671875,
 'LSTM-120-(1)': 331.875,
 'LSTM-120-(2)': 348.73828125,
 'LSTM-120-(3)': 364.86328125,
 'LSTM-200-(1)': 335.2265625,
 'LSTM-200-(2)': 358.38671875,
 'LSTM-200-(3)': 381.08203125,
 'LSTM-500-(1)': 359.86328125,
 'LSTM-500-(2)': 437.7109375,
 'LSTM-500-(3)': 511.9296875,
 'LSTM-1000-(1)': 455.09375,
 'LSTM-1000-(2)': 711.21484375,
 'LSTM-1000-(3)': 968.98046875}

In [14]:
trainable_parameters_dict

{'LSTM-80-(1)': 27925,
 'LSTM-80-(2)': 79445,
 'LSTM-80-(3)': 130965,
 'LSTM-100-(1)': 42905,
 'LSTM-100-(2)': 123305,
 'LSTM-100-(3)': 203705,
 'LSTM-120-(1)': 61085,
 'LSTM-120-(2)': 176765,
 'LSTM-120-(3)': 292445,
 'LSTM-200-(1)': 165805,
 'LSTM-200-(2)': 486605,
 'LSTM-200-(3)': 807405,
 'LSTM-500-(1)': 1014505,
 'LSTM-500-(2)': 3016505,
 'LSTM-500-(3)': 5018505,
 'LSTM-1000-(1)': 4029005,
 'LSTM-1000-(2)': 12033005,
 'LSTM-1000-(3)': 20037005}